In [1]:
import numpy as np
import pandas as pd
import ee
import requests
import os
import geopandas as gpd

ee.Initialize(project='ts-rishimnb')

def get_landsat(start_date,end_date, file_name, bbox):
    bbox=ee.Geometry.Rectangle(bbox)
    image_collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") \
    .filterBounds(bbox) \
    .filterDate(start_date,end_date) \
    
    try:
        img =image_collection.median()
        url = img.getDownloadUrl({
            'bands': [
            "SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B7"],
            'region': bbox,
            'scale': 100,
            'format': 'GEO_TIFF'
        })
        response = requests.get(url)
        with open(file_name, 'wb') as fd:
            fd.write(response.content)
    except:
        print(f"{start_date}  {end_date}")



In [2]:
cities={"Delhi":[77.07301009999999, 28.4041, 77.3449601, 28.6506044],
        "Hyderabad": [78.1599217, 17.2168886, 78.6561694, 17.6078088],
        "Bangalore": [77.3791981, 12.7342888, 77.8826809, 13.173706],
        "Kolkata": [88.233628, 22.4520292, 88.4610775, 22.6188255],
        "Chennai": [80.0817007, 12.8338848, 80.3356458, 13.2611661],
        "Mumbai" : [72.77557589999999, 18.8934053, 72.9823065, 19.2717056],
        "Ahmedabad": [72.4534827, 22.902676, 72.7053737, 23.1378156],
        "Pune": [73.7394779, 18.4134784, 73.9864569, 18.6357545],
        "Kanpur": [80.2087784, 26.3512674, 80.46798710000002, 26.54308],
        "Surat": [72.7013612, 21.0478169, 72.9432106, 21.2705834],
        }

for city in cities:
    for year in range(2000,2014):
        os.makedirs(rf"C:\Users\rishi\ml_projects\UHI\Landsat_{year-2000}\{city}\NDVI",exist_ok=True)

months = {
    1: ("January", 31),
    2: ("February", 28),
    3: ("March", 31),
    4: ("April", 30),
    5: ("May", 31),
    6: ("June", 30),
    7: ("July", 31),
    8: ("August", 31),
    9: ("September", 30),
    10: ("October", 31),
    11: ("November", 30),
    12: ("December", 31)
}

In [3]:
# for city in cities:
#     for year in range(2020,2021):
#         for month in range(4,8):
#             bbox=cities[city]
#             file_name=rf"C:\Users\rishi\ml_projects\UHI\Landsat_{year-2000}\{city}\LST\{months[month][0]}.tif"
#             start_date=f"{year}-{month}-1"
#             print(start_date)
#             end_date=f"{year}-{month}-{months[month][1]}"
#             print(end_date)
#             get_landsat(start_date,end_date,file_name,bbox)

In [4]:
import concurrent.futures

def process_landsat(city, year, month):
    bbox = cities[city]
    file_name = rf"C:\Users\rishi\ml_projects\UHI\Landsat_{year-2000}\{city}\NDVI\{months[month][0]}.tif"
    start_date = f"{year}-{month}-1"
    end_date = f"{year}-{month}-{months[month][1]}"
    
    print(f"Processing {city}, {year}-{month}: {start_date} to {end_date}")
    get_landsat(start_date, end_date, file_name, bbox)

# Use ThreadPoolExecutor to parallelize
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = []
    for city in cities:
        for year in range(2000, 2014):
            for month in range(1,13):
                futures.append(executor.submit(process_landsat, city, year, month))

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


Processing Delhi, 2000-1: 2000-1-1 to 2000-1-31
Processing Delhi, 2000-2: 2000-2-1 to 2000-2-28
Processing Delhi, 2000-3: 2000-3-1 to 2000-3-31
Processing Delhi, 2000-4: 2000-4-1 to 2000-4-30
Processing Delhi, 2000-5: 2000-5-1 to 2000-5-31
Processing Delhi, 2000-6: 2000-6-1 to 2000-6-30
Processing Delhi, 2000-7: 2000-7-1 to 2000-7-31
Processing Delhi, 2000-8: 2000-8-1 to 2000-8-31
Processing Delhi, 2000-9: 2000-9-1 to 2000-9-30
Processing Delhi, 2000-10: 2000-10-1 to 2000-10-31
Processing Delhi, 2000-11: 2000-11-1 to 2000-11-30
Processing Delhi, 2000-12: 2000-12-1 to 2000-12-31
Processing Delhi, 2001-1: 2001-1-1 to 2001-1-31
Processing Delhi, 2001-2: 2001-2-1 to 2001-2-28
Processing Delhi, 2001-3: 2001-3-1 to 2001-3-31
Processing Delhi, 2001-4: 2001-4-1 to 2001-4-30
Processing Delhi, 2001-5: 2001-5-1 to 2001-5-31
Processing Delhi, 2001-6: 2001-6-1 to 2001-6-30
Processing Delhi, 2001-7: 2001-7-1 to 2001-7-31
Processing Delhi, 2001-8: 2001-8-1 to 2001-8-31
2000-1-1  2000-1-31
Processing 